In [ ]:
def register_file_upload_callback(app, name):
    @app.callback(
        Output(f'uploaded-file-{name}', component_property='data'),
        Input(f'upload-data-{name}', component_property='contents'),
        State(f'upload-data-{name}', component_property='filename'),
    )
    def upload_files(contents, filenames):
        print("\n=== Starting File Upload ===")
        print(f"Contents received: {'Yes' if contents else 'No'}")
        print(f"Filenames received: {filenames}")

        if contents is None:
            print("No contents, returning empty lists")
            return [], [], []
            
        if not isinstance(contents, list):
            print("Converting single file to list")
            contents = [contents]
            filenames = [filenames]
            
        print(f"Processing {len(contents)} files")
        
        encoded_files = []
        file_contents = []
        file_names = []
        
        for content, filename in zip(contents, filenames):
            print(f"\nProcessing file: {filename}")
            try:
                # Split content string to get the base64 part
                content_type, content_string = content.split(',')
                print(f"Content type: {content_type}")

                # For PDFs
                if 'pdf' in filename.lower():
                    print("PDF file detected")
                    # Just store the original content for PDFs
                    encoded_files.append(content)
                    file_contents.append("")  # Empty string for PDF text content
                    file_names.append(filename)
                    print(f"Added PDF {filename} to results")
                
                # For text files
                elif 'txt' in filename.lower():
                    print("Text file detected")
                    encoded_files.append(content)
                    file_contents.append(content)
                    file_names.append(filename)
                    print(f"Added text file {filename} to results")

            except Exception as e:
                print(f"Error processing file {filename}: {str(e)}")
                continue

        print("\n=== Upload Results ===")
        print(f"Total files processed: {len(encoded_files)}")
        print(f"Files: {file_names}")
        
        return encoded_files, file_contents, file_names
    
def register_update_tabs_callback(app, name):
    @app.callback(
        Output('file-tabs' + name, component_property='tabs'),
        Output('file-tabs' + name, component_property='children'),
        Input('uploaded-file' + name, component_property='data'),
        Input('file-tabs' + name, component_property='active_tab'),
    )
    def update_tabs(data, active_tab):
        print("\n=== Update Tabs Called ===")
        print(f"Received data: {bool(data)}")
        
        if not data or not data[0]:
            print("No data available")
            return [], []
            
        encoded_files, file_contents, file_names = data
        print(f"Files to display: {file_names}")
        
        tabs = []
        children = []
        
        for i, (file_name, encoded_file) in enumerate(zip(file_names, encoded_files)):
            tab_id = f'tab{i + 1}{name}'
            
            tabs.append({
                'id': tab_id,
                'title': file_name
            })
            
            children.append(
                html.Div([
                    html.Iframe(
                        src=encoded_file,
                        style={
                            'width': '100%',
                            'height': '800px',
                            'border': 'none'
                        }
                    )
                ])
            )
            
            print(f"Added tab for {file_name}")
        
        return tabs, children